# BioExplorer - Neurons
![](../bioexplorer_neurons_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3

url = 'localhost:5000'
be = BioExplorer(url)

core = be.core_api()
status = be.reset_scene()

### Load neurons

In [ ]:
assembly_name = 'Neurons'
be.remove_assembly(assembly_name)
neurons_assembly = be.add_assembly(assembly_name)

neurons_model = be.add_neurons(
    assembly_name=assembly_name,
    population_name='neurons',
    morphology_representation=be.MORPHOLOGY_REPRESENTATION_SEGMENT,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
    realism_level=be.MORPHOLOGY_REALISM_LEVEL_ALL,
    load_synapses=True, generate_varicosities=True,
    load_somas=True, load_axon=True, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=True, generate_externals=True,
    sql_node_filter='guid=49'
)

In [ ]:
status = core.set_camera(
    orientation=[-0.04516827475434093, -0.12648330315414616, 0.002897551396689663, 0.9909356211050424],
    position=[419.9380089738668, 1864.926535721847, 215.43552754741884],
    target=[452.2792946677419, 1853.483701015516, 91.20223249410772]
)
status = core.set_renderer()

In [ ]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name, palette_size):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, palette_size)

    for material_id in material_ids:
        mid = material_id % palette_size
        if mid==be.NEURON_MATERIAL_SYNAPSE:
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.001)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_MITOCHONDRION:
            colors.append([1.0, 0.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.5)
            user_params.append(1.0)
            specular_exponents.append(6.0)
            emissions.append(0.25)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            colors.append([1.0, 1.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(1.0)
            user_params.append(0.2)
            specular_exponents.append(30.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_SOMA:
            colors.append(palette[0])
            opacities.append(0.75)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.2)
            user_params.append(0.01)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_MYELIN_STEATH:
            colors.append([0.4, 0.3, 0.5])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_DIFFUSE)
            glossinesses.append(1.0)
            user_params.append(1.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        else:
            # Membrane
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.1)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            
        refraction_indices.append(1.0)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [ ]:
model_ids = be.get_model_ids()['ids']
offset = model_ids[0]
set_morphology_materials(offset, 'Set3', be.NB_MATERIALS_PER_MORPHOLOGY)
core.set_renderer()

In [ ]:
def set_materials(palette_name):
    model_ids = be.get_model_ids()['ids']
    model_id = model_ids[len(model_ids)-1]
    material_ids = be.get_material_ids(model_id)['ids']
    nb_material_ids = len(material_ids)
    palette = sns.color_palette(palette_name, nb_material_ids)
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    for _ in material_ids:
        shading_modes.append(be.SHADING_MODE_CARTOON)
        user_params.append(3.0)
        glossinesses.append(4.0)
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        shading_modes=shading_modes, user_parameters=user_params,
        glossinesses=glossinesses,
        diffuse_colors=palette, specular_colors=palette)

set_materials('Set2')
core.set_renderer()

In [ ]:
status = core.set_renderer(
    background_color=[1,1,1],
    current='bio_explorer',subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 10.0
params.shadows = 1.0
params.soft_shadows = 1.0
params.epsilon_factor = 40.0
params.max_bounces = 10
params.show_background = True
params.exposure = 1.1
status = core.set_renderer_params(params)
status = core.set_renderer()